In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 20426932224 (19.02 GB)
Present Capacity: 7378034688 (6.87 GB)
DFS Remaining: 7377985536 (6.87 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.27.0.4:9866 (project-4-data4lyf4-dn-1.project-4-data4lyf4_default)
Hostname: 7dc1b469362a
Decommission Status : Normal
Configured Capacity: 10213466112 (9.51 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 6507671552 (6.06 GB)
DFS Remaining: 3688992768 (3.44 GB)
DFS Used%: 0.00%
DFS Remaining%: 36.12%
Configure

In [2]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv
!rm hdma-wi-2021.csv
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

rm: cannot remove 'hdma-wi-2021.csv': No such file or directory
--2023-10-27 22:46:45--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  81.0MB/s    in 2.1s    

2023-10-27 22:46:47 (81.0 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [3]:
!hdfs dfs -D dfs.replication=1 -D dfs.block.size=1m -cp ./hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.replication=2 -D dfs.block.size=1m -cp ./hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [4]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [5]:
#q3
import requests
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=LISTSTATUS")
status = r.json()
total_size = status['FileStatuses']['FileStatus'][0]['length']
block_size = status['FileStatuses']['FileStatus'][0]['blockSize']
fs = status['FileStatuses']
fs['FileStatus'] = fs['FileStatus'][0]
fs

{'FileStatus': [{'accessTime': 1698446810509,
   'blockSize': 1048576,
   'childrenNum': 0,
   'fileId': 16386,
   'group': 'supergroup',
   'length': 174944099,
   'modificationTime': 1698446816121,
   'owner': 'root',
   'pathSuffix': '',
   'permission': '644',
   'replication': 1,
   'storagePolicy': 0,
   'type': 'FILE'}]}

In [6]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()['Location']

'http://7dc1b469362a:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [7]:
#q5
counts = {}
for offset in range(0, total_size, block_size):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(offset))
    url = r.json()['Location']
    id = url[url.find('http://') + len('http://'):url.find(':9864')]
    if id in counts:
        counts[id] += 1
    else:
        counts[id] = 1
counts

{'7dc1b469362a': 92, '4c44021d23eb': 75}

In [8]:
#q6
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

with hdfs.open_input_file("hdfs://boss:9000/single.csv") as f:
    bts = f.read_at(10, 0)

bts

2023-10-27 22:47:17,243 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [9]:
#q7
import io

with hdfs.open_input_file("hdfs://boss:9000/single.csv") as f:
    reader = io.BufferedReader(f)
    cnt = 0
    for line in reader:
        if "Single Family" in line.decode("utf-8"):
            cnt += 1

cnt

444874